In [64]:
import pandas as pd
import os
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from sklearn.neighbors import NearestNeighbors
import pickle
import random
import math
from sklearn.metrics.pairwise import cosine_similarity

In [2]:
random.seed(3214)

In [3]:
with open('../data/pickles/main_dict.pickle', 'rb') as f:
    main_dict = pickle.load(f)

In [4]:
df = pd.read_csv('../data/csv/merge_df.csv', index_col=0)
df.head()

,track,1_mfcc_1,1_mfcc_2,1_mfcc_3,1_mfcc_4,1_mfcc_5,1_mfcc_6,1_mfcc_7,1_mfcc_8,1_mfcc_9,...,all_ss_min,all_ss_std,all_sf_max,all_sf_mean,all_sf_min,all_sf_std,all_zcr_max,all_zcr_mean,all_zcr_min,all_zcr_std
0,061010.mp3,-724.060181,174.672104,-22.694757,5.049683,-10.285789,-7.565227,-17.961994,-1.329323,-13.898221,...,0.737701,0.991601,0.240276,0.041676,0.008984,0.031454,0.108398,0.029397,0.009277,0.014918
1,061172.mp3,-692.448303,117.801537,18.185888,5.736947,5.938968,31.851004,1.060427,4.214683,-0.146251,...,-0.198049,0.948205,0.390652,0.080923,0.000357,0.059621,0.351562,0.068662,0.002930,0.058827
2,061589.mp3,-697.955811,112.710243,80.530632,18.408318,16.321367,33.912514,-1.067550,19.436161,2.497976,...,-0.194649,0.554220,0.488206,0.133584,0.017999,0.079505,0.125000,0.028202,0.000977,0.020876
3,061493.mp3,-630.083130,135.834732,-17.474176,36.666378,20.877747,3.231712,0.163455,-4.143034,-7.589878,...,0.461225,0.344698,0.404520,0.101769,0.022331,0.065322,0.267090,0.083095,0.006348,0.036801
4,061492.mp3,-692.294617,154.660141,-3.271099,31.385326,22.588284,4.178376,-0.790970,-4.321088,-4.325193,...,0.710971,0.538033,0.420857,0.083492,0.010698,0.055768,0.450195,0.098991,0.005859,0.067057


In [5]:
test_idx = list()
while len(test_idx) < 100:
    idx = random.randint(0, df.shape[0] - 1)
    track_id = int(df.loc[idx].track[:-4])
    tags = main_dict[track_id]['all_tags']
    if len(tags) >= 10:
        test_idx.append(idx)

In [6]:
df_test = df.loc[test_idx]
df_test

,track,1_mfcc_1,1_mfcc_2,1_mfcc_3,1_mfcc_4,1_mfcc_5,1_mfcc_6,1_mfcc_7,1_mfcc_8,1_mfcc_9,...,all_ss_min,all_ss_std,all_sf_max,all_sf_mean,all_sf_min,all_sf_std,all_zcr_max,all_zcr_mean,all_zcr_min,all_zcr_std
7301,010673.mp3,-626.681091,132.151154,11.753740,45.711430,-0.209040,10.068515,-0.682384,5.395576,-4.435939,...,0.461337,0.281160,0.329500,0.124018,0.027167,0.045374,0.130859,0.031217,0.003418,0.019365
3931,066643.mp3,-687.945374,141.427277,-16.972485,19.092112,6.906140,3.117708,6.967309,-2.045398,-7.208217,...,-0.111801,0.627290,0.380662,0.080697,0.000165,0.061523,0.282227,0.049509,0.002930,0.025951
5176,000194.mp3,-782.241943,178.044266,-29.330959,18.219608,-14.123279,-10.458302,-16.387886,-5.279102,-16.686710,...,0.276664,0.634153,0.264619,0.043619,0.000129,0.038136,0.169922,0.037896,0.001953,0.017058
80,074421.mp3,-623.853394,122.672737,17.257471,4.505549,0.719744,-2.922961,-5.741470,-1.727011,-18.558123,...,0.421106,0.539590,0.264868,0.082399,0.007031,0.040712,0.224121,0.051835,0.009277,0.032241
7903,131553.mp3,-645.512512,74.718880,12.483846,45.145756,38.911320,16.187067,6.199753,0.013115,-3.431449,...,0.100123,0.391042,0.335317,0.112432,0.000625,0.050604,0.271973,0.079668,0.008789,0.049538
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3708,025603.mp3,-705.887451,168.620865,34.891018,33.937485,15.525203,2.845042,-8.320030,2.128073,10.097413,...,0.422947,1.021024,0.493078,0.113007,0.003961,0.076067,0.147949,0.026009,0.002441,0.020538
3319,075417.mp3,-637.247192,73.643715,17.631123,33.222134,11.122272,9.398943,12.663881,11.582001,10.201411,...,0.267713,0.341603,0.467992,0.120776,0.001821,0.087565,0.348633,0.081037,0.009766,0.044586
5911,125154.mp3,-694.168884,92.781448,59.304184,42.092861,12.254707,19.374973,4.323670,7.782975,-5.282559,...,-0.067733,0.774724,0.482886,0.110169,0.003097,0.075931,0.317871,0.064865,0.002441,0.054606
3341,075436.mp3,-800.051819,241.068558,40.636051,-0.057632,6.397528,-18.648108,-1.818632,3.204657,-0.186070,...,0.923011,1.546881,1.210505,0.112725,0.010022,0.145565,0.138184,0.015291,0.000488,0.012997


In [7]:
df_train = df.loc[list(set(df.index.values) - set(test_idx))]
df_train

,track,1_mfcc_1,1_mfcc_2,1_mfcc_3,1_mfcc_4,1_mfcc_5,1_mfcc_6,1_mfcc_7,1_mfcc_8,1_mfcc_9,...,all_ss_min,all_ss_std,all_sf_max,all_sf_mean,all_sf_min,all_sf_std,all_zcr_max,all_zcr_mean,all_zcr_min,all_zcr_std
0,061010.mp3,-724.060181,174.672104,-22.694757,5.049683,-10.285789,-7.565227,-17.961994,-1.329323,-13.898221,...,0.737701,0.991601,0.240276,0.041676,8.983906e-03,0.031454,0.108398,0.029397,0.009277,0.014918
2,061589.mp3,-697.955811,112.710243,80.530632,18.408318,16.321367,33.912514,-1.067550,19.436161,2.497976,...,-0.194649,0.554220,0.488206,0.133584,1.799921e-02,0.079505,0.125000,0.028202,0.000977,0.020876
3,061493.mp3,-630.083130,135.834732,-17.474176,36.666378,20.877747,3.231712,0.163455,-4.143034,-7.589878,...,0.461225,0.344698,0.404520,0.101769,2.233087e-02,0.065322,0.267090,0.083095,0.006348,0.036801
4,061492.mp3,-692.294617,154.660141,-3.271099,31.385326,22.588284,4.178376,-0.790970,-4.321088,-4.325193,...,0.710971,0.538033,0.420857,0.083492,1.069842e-02,0.055768,0.450195,0.098991,0.005859,0.067057
5,061592.mp3,-708.504639,136.741913,9.762825,40.804737,17.343578,28.947464,15.307942,17.034895,9.786160,...,-0.039187,0.629073,0.458335,0.132431,6.568079e-11,0.090427,0.515137,0.018989,0.002441,0.039541
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7977,131914.mp3,-654.926941,144.197983,-5.618096,12.815182,-4.743158,4.794647,-0.457318,2.406655,-8.439367,...,0.907090,0.585478,0.172700,0.063363,1.671647e-02,0.027547,0.136230,0.058892,0.012207,0.026086
7978,131772.mp3,-625.237183,158.683304,-10.997794,23.054964,11.636267,-1.839731,-17.972336,-9.967291,-12.986640,...,0.608825,0.747877,0.245719,0.111260,6.102545e-03,0.040364,0.070312,0.020365,0.005859,0.009048
7979,131792.mp3,-655.022644,144.006958,5.970843,14.379925,14.236056,3.561646,-4.041969,-4.800819,-3.666181,...,0.740162,0.707120,0.290256,0.078840,7.514172e-03,0.051776,0.104980,0.032318,0.000488,0.014633
7980,131624.mp3,-788.494507,75.723251,35.248608,28.392780,4.289547,18.946749,-5.460835,10.575089,-8.978883,...,0.130618,0.542250,0.212018,0.045943,9.695412e-04,0.043634,0.321289,0.102740,0.007812,0.053932


In [9]:
features_train = df_train.drop(['track'], 1)
features_test = df_test.drop(['track'], 1)

In [10]:
scaler = StandardScaler()
scaler.fit(features_train)
X_train = scaler.transform(features_train)
X_test = scaler.transform(features_test)

In [11]:
pca = PCA(0.95)
pca.fit(X_train)
X_train = pca.transform(X_train)
X_test = pca.transform(X_test)

In [12]:
X_train.shape

(7885, 103)

In [13]:
X_test.shape

(100, 103)

In [100]:
neigh = NearestNeighbors(10, metric='euclidean')
neigh.fit(X_train)

NearestNeighbors(algorithm='auto', leaf_size=30, metric='euclidean',
                 metric_params=None, n_jobs=None, n_neighbors=10, p=2,
                 radius=1.0)

In [58]:
def count_good_recommend(query_idx, answer_list, trhold):
    counter = 0
    query_id = int(df_test.iloc[query_idx].track[:-4])
    relevant = [query_id]
    query_tags = main_dict[query_id]['all_tags']
    for idx in answer_list:
        answer_id = int(df_train.iloc[idx].track[:-4])
        answer_tags = main_dict[answer_id]['all_tags']
        score = 0
        i = 0
        for tag in answer_tags:
            if tag in query_tags:
                i += 1
        if i == 0:
            continue
        
        precision = i / len(answer_tags)
        recall = i / len(query_tags)
        score = 2 * precision * recall / (precision + recall)
        if score >= trhold:        
            counter += 1
            relevant.append(answer_id)
    return counter, relevant

In [59]:
def count_ndcg(neighbors, relevant_list):
    dcg = 0
    idcg = 0
    for i in range(len(neighbors)):
        track = int(df_train.iloc[neighbors[i]].track[:-4])
        if track in relevant_list:
            dcg += 1 / math.log(i + 2, 2)
        idcg += 1 / math.log(i + 2, 2)
    ndcg = dcg / idcg
    return ndcg

In [101]:
mean_ap = 0
ndcg = 0
for i in range(100):
    neighbors = neigh.kneighbors([X_test[i]])[1][0]
    relevant_num, relevant_list = count_good_recommend(i, neighbors, trhold=0.4)
#     print(relevant_list)

    p_at_10 = relevant_num / 10
    mean_ap += p_at_10
    ndcg += count_ndcg(neighbors, relevant_list[1:])
mean_ap /= 100 
mean_ndcg = ndcg / 100

In [102]:
mean_ap

0.09299999999999997

In [103]:
mean_ndcg

0.10454909808110224

In [110]:
cos_similarity_mtx = cosine_similarity(X_test, X_train)

In [111]:
mean_ap = 0
ndcg = 0
for i in range(100):
    arr = cos_similarity_mtx[i]
    neighbors = arr.argsort()[-10:][::-1]
    relevant_num, relevant_list = count_good_recommend(i, neighbors, trhold=0.4)
#     print(relevant_list)
    print(relevant_list[0], [df_train.iloc[x].track[:-4] for x in neighbors])
    p_at_10 = relevant_num / 10
    mean_ap += p_at_10
    ndcg += count_ndcg(neighbors, relevant_list[1:])
mean_ap /= 100 
mean_ndcg = ndcg / 100

10673 ['028480', '141874', '030682', '074908', '050836', '111311', '036481', '030041', '095564', '111873']
66643 ['091468', '149101', '142575', '131901', '142574', '122082', '142564', '126105', '088863', '146150']
194 ['000211', '127870', '046238', '061010', '006331', '038779', '145887', '128885', '043028', '120309']
74421 ['148214', '148215', '020704', '052128', '085291', '045514', '142529', '130944', '042146', '106502']
131553 ['029961', '123835', '064522', '126104', '042475', '129046', '087641', '052645', '055826', '055831']
24431 ['036987', '024422', '066638', '024425', '036994', '051260', '052141', '114037', '114042', '056804']
144469 ['095724', '095726', '091934', '127037', '122646', '017462', '136995', '106564', '126455', '144475']
87100 ['087106', '144476', '139638', '133975', '093917', '131917', '121655', '134586', '108023', '133538']
69204 ['035549', '055242', '114384', '042017', '150078', '114385', '139937', '104283', '133022', '150080']
27454 ['026655', '122683', '141878', 

85027 ['107592', '116709', '116456', '004450', '139773', '118063', '084195', '128471', '123485', '069188']
127720 ['110983', '117450', '038904', '125777', '086263', '133272', '042044', '067500', '118987', '122472']
40851 ['114293', '145758', '040683', '145257', '064334', '075937', '085041', '085828', '075755', '106277']
1686 ['035608', '069792', '004080', '118489', '019891', '004234', '055097', '105917', '019889', '047100']
73365 ['140584', '081576', '141297', '113028', '046158', '069822', '135989', '107127', '073367', '027855']
5264 ['083715', '149138', '032325', '025668', '021404', '113512', '144182', '117631', '146478', '071709']
19759 ['019758', '098299', '134937', '075869', '130134', '081083', '115269', '036277', '117666', '149523']
41568 ['125825', '122936', '127663', '036643', '056797', '141568', '083512', '096697', '078156', '107567']
24429 ['119545', '110690', '146019', '097960', '116547', '113789', '069784', '025804', '052449', '141594']
86788 ['045154', '096401', '147412', '

In [106]:
mean_ap

0.09299999999999997

In [107]:
mean_ndcg

0.1070406809817999

In [108]:
for i in range(100):
    arr = cos_similarity_mtx[i]
    cos_neighbors = arr.argsort()[-1:][::-1]
    euclid_neighbors = neigh.kneighbors([X_test[i]])[1][0]
    test = df_test.iloc[i].track
    euclid_recom = df_train.iloc[euclid_neighbors[0]].track
    cos_recom = df_train.iloc[cos_neighbors[0]].track

    print(test, euclid_recom, cos_recom)

010673.mp3 030682.mp3 028480.mp3
066643.mp3 091468.mp3 091468.mp3
000194.mp3 006331.mp3 000211.mp3
074421.mp3 148214.mp3 148214.mp3
131553.mp3 123835.mp3 029961.mp3
024431.mp3 024422.mp3 036987.mp3
144469.mp3 091934.mp3 095724.mp3
087100.mp3 087106.mp3 087106.mp3
069204.mp3 006439.mp3 035549.mp3
027454.mp3 026655.mp3 026655.mp3
004835.mp3 055430.mp3 059721.mp3
000667.mp3 049401.mp3 049401.mp3
014063.mp3 072072.mp3 149452.mp3
137171.mp3 119828.mp3 119828.mp3
061172.mp3 011868.mp3 097814.mp3
107579.mp3 073170.mp3 073170.mp3
137171.mp3 119828.mp3 119828.mp3
024901.mp3 117057.mp3 012737.mp3
048463.mp3 025378.mp3 045102.mp3
145550.mp3 064249.mp3 091790.mp3
011786.mp3 124917.mp3 124917.mp3
073764.mp3 017499.mp3 070655.mp3
012394.mp3 066539.mp3 084157.mp3
073761.mp3 082917.mp3 082917.mp3
108962.mp3 127299.mp3 127299.mp3
000190.mp3 000621.mp3 038833.mp3
062337.mp3 091851.mp3 004799.mp3
064093.mp3 145750.mp3 139933.mp3
004233.mp3 011682.mp3 011682.mp3
075752.mp3 109670.mp3 110070.mp3
124702.mp3

In [67]:
cos_similarity_mtx = cosine_similarity(X_test, X_train)

In [73]:
arr = cos_similarity_mtx[0]

In [76]:
arr.argsort()[-10:][::-1]

array([ 659, 6443, 2378,   71, 1067, 7035, 2826, 2379, 7715, 7009])

In [84]:
cos_similarity_mtx[0].sort()

In [85]:
cos_similarity_mtx[0][-10:]

array([0.76576922, 0.77147324, 0.78109024, 0.78396155, 0.78587406,
       0.78773973, 0.80250647, 0.80814343, 0.81528035, 0.83231566])

In [87]:
math.cos(0)

1.0

In [22]:
print(f'euclidean meanAP {mean_ap}, meanNDCG {mean_ndcg}')

euclidean meanAP 0.09299999999999997, meanNDCG 0.10454909808110224


In [29]:
print(f'namhattan meanAP {mean_ap}, meanNDCG {mean_ndcg}')

namhattan meanAP 0.06899999999999998, meanNDCG 0.08812867556608356


In [36]:
print(f'chebyshev meanAP {mean_ap}, meanNDCG {mean_ndcg}')

chebyshev meanAP 0.05899999999999998, meanNDCG 0.0732865384316484


In [45]:
print(f'mahalanobis meanAP {mean_ap}, meanNDCG {mean_ndcg}')

mahalanobis meanAP 0.04799999999999999, meanNDCG 0.056724564869462514
